In [1]:
import pickle
import re
import os
import time

os.chdir('/Users/Bya/git/predictEPL/MyFunctions/')
import tokenizers
import senti_word_net
import featx
import replacers
import dataIO
import converter

from nltk.corpus import stopwords

from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

os.chdir('/Users/Bya/Dropbox/Research/datas/movieclassifier/')
from vectorizer import vect


# load Long IMDB model
clf = pickle.load(open(
        os.path.join('pkl_objects',
                     'classifier.pkl'), 'rb'))

# load Short IMDB model
with open("/Users/Bya/git/predictEPL/NLTK/NLTK_TUTORIAL/pickled_algos/naiveBayes_for_short_reviews_NEW.pickle", "rb") as saved_classifier_f:
    nb_classifier = pickle.load(saved_classifier_f)

def cleanHash(word):
    if word[0] == '#':
        return '#'
    elif word[0] == '@':
        return '@'
    elif word[0:4] == 'http':
        return 'http'
    else:
        return word

def sentiment_short_long_IMDB(tweet):
    
    # can't -> cannot, bya's -> bya is
    replacer = replacers.RegexpReplacer()
    tweet = replacer.replace(tweet)
    
    # Tweet tokenizer
    words = TweetTokenizer().tokenize(tweet)
    
    # defining stopwords
    english_stops = set(stopwords.words('english'))
    english_stops_added = english_stops | {'.', ',', ':', ';', '#', '?', 'RT', '-', '@', 'rt'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
    
    # Stemmer
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    
    # words = map(lambda word: cleanHash(word), words)
    words = [cleanHash(word) for word in words]
    
    # defining stopwords
    english_stops_added =  {'.', ',', '!', ':', ';', '#', '?', 'RT', '-', '@', 'rt', 'http'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
 

    # Naive Bayes: IMDB Short Reviews   
    probs = nb_classifier.prob_classify(featx.bag_of_words(words))
    short_sent = probs.max()
    short_pos = probs.prob('pos')
    short_neg = probs.prob('neg')
    
    
    # Naive Bayes: IMDB Long Reviews
    label = {0: 'neg', 1:'pos'}
    sentence = [' '.join(words)]
    X = vect.transform(sentence)
    long_sent = label[clf.predict(X)[0]]
    long_pos = clf.predict_proba(X)[0][1]
    long_neg = clf.predict_proba(X)[0][0]
    
    
    return (short_sent, short_pos, short_neg,
            long_sent, long_pos, long_neg)

In [12]:
# Change home directory
pathData = "/Users/Bya/Dropbox/Research/datas/"
os.chdir(pathData)

filenames = dataIO.GetFilenames('GW' + str(10) + '/SingleGames')
filenames

['GW10/SingleGames/Arsenal_vs_Everton.csv',
 'GW10/SingleGames/Bournemouth_vs_Tottenham.csv',
 'GW10/SingleGames/Leicester_vs_Crystal.csv',
 'GW10/SingleGames/Liverpool_vs_Southampton.csv',
 'GW10/SingleGames/Norwich_vs_WestBromwich.csv',
 'GW10/SingleGames/Stoke_vs_Watford.csv',
 'GW10/SingleGames/Sunderland_vs_Newcastle.csv',
 'GW10/SingleGames/United_vs_City.csv',
 'GW10/SingleGames/Villa_vs_Swansea.csv',
 'GW10/SingleGames/WestHam_vs_Chelsea.csv']

In [13]:
for filePath in filenames:
    start_time = time.time()

    GW = filePath.split('/')[0]
    home_team = filePath[:-4].split('/')[2].split('_')[0]
    away_team = filePath[:-4].split('/')[2].split('_')[2]
    print(GW, home_team, away_team)

    # Change home directory
    pathData = "/Users/Bya/Dropbox/Research/datas/"
    os.chdir(pathData)    

    # read csv as df
    df = dataIO.csv_dic_df(filePath)

    # add 'time' column
    df['time'] = df['date']
    df.time = [converter.toSeconds(date) for date in df.time]

    # set time
    start = converter.toSeconds(df.date[0]) - int(df['date'][0][-13:-11])

    # add 'ith_minute'
    df['ith_minute'] = (df['time'] - start) / 60

    short_sent = []
    short_pos = []
    short_neg = []
    long_sent = []
    long_pos = []
    long_neg = []

    for row in df.iterrows():
        tweet = row[1]['text']
        x,y,z,a,b,c = sentiment_short_long_IMDB(tweet)
        short_sent.append(x)
        short_pos.append(y)
        short_neg.append(z)
        long_sent.append(a)
        long_pos.append(b)
        long_neg.append(c)

    df['short_sent'] = short_sent
    df['short_pos'] = short_pos
    df['short_neg'] = short_neg
    df['long_sent'] = long_sent
    df['long_pos'] = long_pos
    df['long_neg'] = long_neg  


    savePath = '/Users/Bya/Dropbox/Research/datas/Results/NB_Short_Long/' + GW
    dataIO.DFtoCSV(df, savePath, home_team + "_vs_" + away_team, False)

    # print computing time
    print(time.time() - start_time)

GW10 Arsenal Everton
20.10829520225525
GW10 Bournemouth Tottenham
11.920794010162354
GW10 Leicester Crystal
4.213562965393066
GW10 Liverpool Southampton
29.66409993171692
GW10 Norwich WestBromwich
3.836544990539551
GW10 Stoke Watford
4.5339131355285645
GW10 Sunderland Newcastle
16.681102991104126
GW10 United City
76.45412611961365
GW10 Villa Swansea
8.870604038238525
GW10 WestHam Chelsea
35.87384510040283
